In [1]:
from pathlib import Path

docs_dir = Path(
    "/Users/f006pfk/Dropbox (Dartmouth College)/RR-workshop-data/federalist-papers/split"
)

In [2]:
from langchain.document_loaders import DirectoryLoader

doc_loader = DirectoryLoader(docs_dir, show_progress=True, recursive=False)
docs = doc_loader.load()

100%|██████████| 85/85 [00:09<00:00,  8.89it/s]


In [6]:
from langchain.llms import GPT4All
from pathlib import Path

model_path = Path(
    "~/shared/RR-workshop-data/pretrained-models/llm/gpt4all-falcon-q4_0.gguf"
).expanduser()

llm = GPT4All(model=str(model_path))

In [7]:
from langchain.text_splitter import CharacterTextSplitter

In [8]:
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=500, chunk_overlap=0
)
texts = text_splitter.split_documents(docs[:1])

In [9]:
from langchain.chains.summarize import load_summarize_chain

chain = load_summarize_chain(llm=llm, chain_type="map_reduce")

chain.run(texts)